In [30]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [31]:
import awswrangler as wr

import mlflow

# Para que funcione, en todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio
%env AWS_SECRET_ACCESS_KEY=minio123
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [32]:
mlflow_server = "http://localhost:5001"

mlflow.set_tracking_uri(mlflow_server)

In [33]:
import datetime
from os import getenv
import warnings

import boto3
import json
import matplotlib.pyplot as plt
from mlflow.models import infer_signature
from neuralprophet import NeuralProphet
import numpy as np
import optuna
from sklearn.metrics import mean_absolute_error

from mlflow_aux import get_or_create_experiment
from neural_prophet_wrapper import NeuralProphetWrapper
from optuna_aux import champion_callback, objective
from trainer_aux import set_trainer

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

warnings.filterwarnings("ignore")

### **Model training**

In [34]:
raw_data_path = "s3://data/raw/trx_recaudacion_km_empresa_54_2020_2025.xlsx"
sheets = wr.s3.read_excel(raw_data_path, sheet_name=None)
lines = sheets.keys()

N_TRIALS = 30
EPOCHS = 100
BATCH_SIZE = 128
regressors = ["km"]

model_lines = {}

for line in lines:

    # Read training and validation data
    df_train = wr.s3.read_csv(f"s3://data/processed/{line}/train.csv", parse_dates=["ds"])
    df_test = wr.s3.read_csv(f"s3://data/processed/{line}/test.csv", parse_dates=["ds"])

    experiment_id = get_or_create_experiment(f"Line {line}")

    run_name_parent = "best_hyperparam_" + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):

        # Hyperparameter optimization with Optuna
        study = optuna.create_study(direction="minimize")
        study.optimize(
            lambda trial: objective(
                trial, df_train.copy(), df_test.copy(),
                EPOCHS, BATCH_SIZE, regressors
            ),
            n_trials=N_TRIALS,
            callbacks=[champion_callback]
        )

        # Log best hyperparameters
        mlflow.log_params(study.best_params)
        mlflow.log_metric("best_train_mae", study.best_value)

        mlflow.set_tags({
            "project": f"Line {line}",
            "optimizer_engine": "optuna",
            "model_family": "neuralprophet",
            "feature_set_version": 1,
        })

        # Create model with best hyperparameters
        model = NeuralProphet(
            n_changepoints=study.best_params["n_changepoints"],
            trend_reg=study.best_params["trend_reg"],
            learning_rate=study.best_params["learning_rate"],
            seasonality_mode=study.best_params["seasonality_mode"],
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
        )
        set_trainer(model, EPOCHS)

        # Add regressors
        for reg in regressors:
            model.add_future_regressor(reg)
        
        # Train
        model.fit(df_train, freq='D')

        # Predict on validation
        forecast_test = model.predict(df_test)
        # Select only numeric columns
        num_cols = forecast_test.select_dtypes(include=[np.number]).columns

        # Replace negatives with 0 only in numeric columns
        forecast_test[num_cols] = forecast_test[num_cols].clip(lower=0)

        # Error metrics calculation
        mae = mean_absolute_error(df_test['y'].values, forecast_test['yhat1'].values)
        mlflow.log_metric("test_mae", mae)

        # Save model artifact
        artifact_path = f"line_{line}_model"
        sample_input = df_train[["ds", "y"] + regressors].copy().tail(30)
        sample_output = model.predict(sample_input)[["ds", "yhat1"]]
        signature = infer_signature(sample_input, sample_output)

        mlflow.pyfunc.log_model(
            python_model=NeuralProphetWrapper(model),
            code_paths=["./neural_prophet_wrapper.py"],
            name=artifact_path,
            signature=signature,
            registered_model_name=f"line_{line}_model_prod",
            metadata={"model_data_version": 1},
        )

        # --- Forecast on train and test ---
        forecast_train = model.predict(df_train)
        # Select only numeric columns
        num_cols = forecast_train.select_dtypes(include=[np.number]).columns

        # Replace negatives with 0 only in numeric columns
        forecast_train[num_cols] = forecast_train[num_cols].clip(lower=0)

        fig, ax = plt.subplots(figsize=(12, 6))

        # Training actual values
        ax.plot(df_train["ds"], df_train["y"], label="Train Real", color="blue")

        # Training predictions
        ax.plot(forecast_train["ds"], forecast_train["yhat1"], 
                label="Train Pred", color="cyan", linestyle="--")

        # Test actual values
        ax.plot(df_test["ds"], df_test["y"], label="Test Real", color="black")

        # Test predictions
        ax.plot(forecast_test["ds"], forecast_test["yhat1"], 
                label="Test Pred", color="red", linestyle="--")

        ax.set_title(f"Train/Test Forecast - Line {line}")
        ax.set_xlabel("Date")
        ax.set_ylabel("y")
        ax.legend()

        # Save in MLflow
        plot_path = f"/tmp/forecast_train_test_{line}.png"
        fig.savefig(plot_path)
        plt.close(fig)

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # --- Components in train and test ---
        model.plot_components(forecast_train, plotting_backend="matplotlib")
        fig_components = plt.gcf()
        components_path = f"/tmp/components_train_{line}.png"
        fig_components.savefig(components_path)
        plt.close(fig_components)
        mlflow.log_artifact(components_path, artifact_path="plots")

        model.plot_components(forecast_test, plotting_backend="matplotlib")
        fig_components = plt.gcf()
        components_path = f"/tmp/components_test_{line}.png"
        fig_components.savefig(components_path)
        plt.close(fig_components)
        mlflow.log_artifact(components_path, artifact_path="plots")

        # Get the stored model location in MLflow
        model_lines[line] = (mlflow.get_artifact_uri(artifact_path), model, mae)

    mlflow.end_run()

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.


Epoch 100: 100%|██████████| 100/100 [00:00<00:00, 4199.39it/s, loss=0.0345, v_num=2757, MAE=5.35e+3, RMSE=6.59e+3, Loss=0.0346, RegLoss=0.000]  

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.603% of the data.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.603% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 443.47it/s]

...

Registered model 'line_504I_model_prod' already exists. Creating a new version of this model...
2025/08/24 20:14:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: line_504I_model_prod, version 4
Created version '4' of model 'line_504I_model_prod'.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 225.48it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.
WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.


🏃 View run best_hyperparam_2025/08/24-20:13:16" at: http://localhost:5001/#/experiments/18/runs/7dc3435c63684996ab3989b27f59d954
🧪 View experiment at: http://localhost:5001/#/experiments/18


### **Upload lines with trained model**

In [35]:
s3 = boto3.client(
    "s3",
    endpoint_url=getenv("AWS_ENDPOINT_URL_S3", default=""),
    aws_access_key_id=getenv("AWS_ACCESS_KEY_ID", default=""),
    aws_secret_access_key=getenv("AWS_SECRET_ACCESS_KEY", default="")
)

bucket_name = "mlflow"
object_key = "lines/avalilable_lines.json"

s3.put_object(
    Bucket=bucket_name,
    Key=object_key,
    Body=json.dumps(list(model_lines.keys())),
    ContentType="application/json"
)

{'ResponseMetadata': {'RequestId': '185ED6FC71438376',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"5c240d6de3ebffde649e7f894370c244"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-checksum-crc32': 'r9qqGg==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '185ED6FC71438376',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '1716',
   'x-ratelimit-remaining': '1716',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 24 Aug 2025 23:14:44 GMT'},
  'RetryAttempts': 0},
 'ETag': '"5c240d6de3ebffde649e7f894370c244"',
 'ChecksumCRC32': 'r9qqGg==',
 'ChecksumType': 'FULL_OBJECT'}